# Find Stars with Photutils &#x2728;
The main goal of this part of the workshop is to detect point sources (some of which will be stars, black holes, neutron stars, star clusters, Active Galactic Nuclei, you name it!) in an astronomical image with the `DAOStarFinder` algorithm and then extract their photometry.

## A little about the algorithm in play

1. Estimating FWHM (`RadialProfile` or `fit_fwhm`)
2. Background Estimation
3. Point Source Detection (Finding stars)
4. Aperture Photometry
5. Your turn!
6. Use case of DaoStarFinder

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# pip install astropy           # uncomment to install astropy and photutils
# pip install photutils

# 1. Estimating FWHM

## using `RadialProfile`
measuring surface brightness against increasing aperture radius

In [ ]:
from astropy.modeling.models import Gaussian2D              # to create a 2D gaussian
from astropy.visualization import simple_norm               # for image normalization
from photutils.datasets import make_noise_image             # to create a noisy image

In [ ]:
# create an artificial single source
gmodel = Gaussian2D(amplitude=42.1,
                    x_mean=47.8, 
                    y_mean=52.4, 
                    x_stddev=4.7, 
                    y_stddev=4.7, 
                    theta=0)       # rotational angle
yy, xx = np.mgrid[0:100, 0:100]
data = gmodel(xx, yy)
bkg_sig = 2.4
noise = make_noise_image(data.shape, mean=0., stddev=bkg_sig, seed=123)

# Add noise to the data
data += noise
error = np.zeros_like(data) + bkg_sig

norm = simple_norm(data, 'sqrt')
plt.imshow(data, origin='lower', norm=norm)

In [ ]:
from photutils.centroids import centroid_quadratic          # to get the centroid position of the gaussian
from photutils.profiles import RadialProfile                

# find the source centroid
xycen = centroid_quadratic(data, xpeak=48, ypeak=52)

# create the radial profile
edge_radii = np.arange(26)
rp = RadialProfile(data, xycen, edge_radii, error=error, mask=None)

norm = simple_norm(data, 'sqrt')
fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(data, norm=norm, origin='lower')
rp.apertures[5].plot(ax=ax, color='C0', lw=2)
rp.apertures[10].plot(ax=ax, color='C1', lw=2)
rp.apertures[15].plot(ax=ax, color='C3', lw=2)
rp.apertures[24].plot(ax=ax, color='C4', lw=2)
plt.show()

In [ ]:
# plot the radial profile
fig, ax = plt.subplots(figsize=(8, 6))
rp.plot(ax=ax, color='C0')
rp.plot_error(ax=ax)
ax.scatter(rp.data_radius, rp.data_profile, s=1, color='C1')
plt.show()

## using `fit_fwhm`

In [ ]:
from photutils.psf import fit_fwhm

# Find the fwhm
fwhm = fit_fwhm(data, fit_shape=99)
print(f"fwhm is {fwhm}")

# 2. Background Estimation

Photutils provides some tools for estimating the background and the background noise in astronomical data. Background as well as background noise estimation are both very difficult tasks but are also required for a variety of purposes:
1. to measure the photometry of an astronomical source
2. to measure the morphological properties (shapes and structures) of astronomical sources
3. to determine the significance of the objects that you find in an astrnomical image
4. for estimating photometric errors of the those objects

For today's workshop, we are mostly interested in points (1), (3) and (4)

## Sigma Clipping sources

It is a reliable and the most widely used technique for background estimation due to the calculations involved. It is used for images where the background and the background noise are relatively constant throughout the image. Pixels that are above or below a specified sigma level from the median are discarded and the statistics are recalculated.

In [ ]:
from photutils.datasets import make_100gaussians_image
from astropy.stats import sigma_clipped_stats

In [ ]:
data = make_100gaussians_image(noise=True)
mean, median, std = sigma_clipped_stats(data, sigma=3.0)   # std is important for the right threshold when detecting point sources
print(f"Mean: {mean}\nMedian: {median}\nStd: {std}")      
plt.imshow(data, origin='lower', vmax=40)
plt.show()

## 2D Background and Noise Estimation

If the background or the background noise is not constant throughout the image, which is what you see with galactic datasets, we generate a 2D image of the background and the background RMS (aka noise). 

In [ ]:
from astropy.stats import SigmaClip
from photutils.background import Background2D, MedianBackground

sigma_clip = SigmaClip(sigma=3.0)    # will iterate over the data, each time rejecting values
                                     # that are less or more than the specified std
bkg_estimator = MedianBackground()   # Calculate background in an array

# Estimate background and background noise
bkg = Background2D(data, (50, 50), filter_size=(3, 3),
                   sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 10))
ax[0].imshow(data, origin='lower', vmax=40, cmap='Greys')
ax[0].set_title('Original image')
ax[1].imshow(data-bkg.background, origin='lower', cmap='Greys', interpolation='nearest', vmax=40)
ax[1].set_title('Background subtracted image')
plt.show()

This was just a few of the many options for background subtraction provided by photutils. For more information on the available options for background subtraction, see this [page](https://photutils.readthedocs.io/en/stable/user_guide/background.html#d-background-and-noise-estimation). 

# 3. Point Source Identification

Now, that we have the fwhm and we know how to estimate the background, we have the tools needed for point source identification. Let's see this with an example image.

In [ ]:
from photutils.datasets import load_star_image

image = load_star_image()

In [ ]:
plt.imshow(image.data, origin='lower', interpolation='nearest')

My general strategy with point source identification of all the sources in the galaxy is to perform the point source identifcation on the subset of the image and see if the code identifies all the obvious sources in the data.

In [ ]:
# Look at subset image
data = image.data[0:401, 0:401]
plt.imshow(data, origin='lower')

In [ ]:
from photutils.detection import DAOStarFinder

mean, median, std = sigma_clipped_stats(data, sigma=3.0)       # can vary sigma depending upon the data
fwhm = 3.0
threshold= 3 * std
daofind = DAOStarFinder(fwhm=fwhm, threshold=threshold)
sources = daofind(data)
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
print(f"Found {len(positions)} sources...")

There are many options available for the shapes of apertures. For more info on the apertures available, see this  [page](https://photutils.readthedocs.io/en/stable/user_guide/aperture.html#apertures).

In [ ]:
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils.aperture import CircularAperture

apertures = CircularAperture(positions, r=4.0)
norm = ImageNormalize(stretch=SqrtStretch())
plt.imshow(data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')
apertures.plot(color='blue', lw=1.5, alpha=0.5)
plt.show()

# 4. Aperture Photometry

Now, while we are at it and we have found the point sources in the image, we might as well get their photometry as well...

In [ ]:
# Generate a background substracted image
sigma_clip = SigmaClip(sigma=5.)
bkg_estimator = MedianBackground()
bkg = Background2D(data, (50, 50), filter_size=(3, 3),
                   sigma_clip=sigma_clip,
                   bkg_estimator=bkg_estimator)
data_sub = data - bkg.background

# Take a look at the backgroudn subtracted data
plt.imshow(data_sub, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')

A note on aperture photometry before we move on...

In [ ]:
from starfind import make_aperture_image

make_aperture_image()

In [ ]:
from photutils.aperture import aperture_photometry

ap_radii = [i for i in range(1, 31)]
apertures = [CircularAperture(positions, r=r) for r in ap_radii]

# always use background subtracted data when doing photometry
photometry = aperture_photometry(data_sub, apertures)

display(photometry)

# 5. Your turn! 

Now, this is your chance to implement the `DAOStarFinder` algorithm in another example image. 

Use the function `find_stars` below to properly detect most (if not all) the point sources in the image. Try playing around with the `fwhm` and the `std_multiple` to see how they affect your results. If the code still does not correctly identify the obvious point sources, try tweaking the `threshold` parameter then. 

In [ ]:
def find_stars(data,
               fwhm,
               threshold=None, 
               sigma=5, 
               radius=5, 
               cmap='gray_r', 
               vmin=0, 
               vmax=10, 
               std_multiple=5, 
               aperture_color='#0547f9', 
               plot=True,
               photometry=True):
    """
    Find point sources in an image using the DAOStarFinder algorithm and then plot apertures 
    around the objects. If `photometry=True`, the code will return the photometric measurements as well.

    PARAMETERS
    ----------
    data : data
        Data
    fwhm : float
        Full Width at Half Maximum of a star in arcseconds. 
        This code will automatically convert the fwhm from 
        arcseconds to pixels.
    threshold : float
        If specified, it will override the threshold to be
        used for identifying stars. Otherwise, the threshold
        is used as a multiple of the standard deviation found
        through the sigma clipped statistics on the data.
    sigma : int, default=5
        sigma to be used for finding the standard deviation.
    conversion : float, default=0.031
        The convesion from arcseconds to pixel.
        Default is 0.031 for JWST NIRCam short wavelength
        filters.
    radius : float, default=5 
        Radius of the aperture to be applied on the sources
        detected. Default is 5.
    cmap : string, default='viridis'
        Colour map to be used when plotting the image. Default 
        is viridis. 
    aperture_color : string, default='limegreen'
        The color of the apertures.

    RETURN
    ------
    positions : np.arrays
        The positions of the point sources.
        Also plots the apertures around the point 
        sources on the image. 
        """
    mean, median, std = sigma_clipped_stats(data, sigma=sigma)

    # If threshold is provided manually.
    if threshold:
        daofind = DAOStarFinder(fwhm=fwhm, threshold=threshold)
        objects = daofind(data)
    else:
        daofind = DAOStarFinder(fwhm=fwhm, threshold=std_multiple*std)
        objects = daofind(data)

    print("Found", len(objects), "objects.")
    positions = np.transpose((objects["xcentroid"], objects["ycentroid"]))

    # Create apertures around sources
    if plot:
        apertures = CircularAperture(positions, r=radius)
        plt.imshow(data,
                   cmap=cmap, 
                   vmin=vmin, 
                   vmax=vmax, 
                   origin='lower',
                   interpolation='nearest')
        apertures.plot(color=aperture_color)
        plt.show()

    if photometry:
        # Generate a background substracted image
        sigma_clip = SigmaClip(sigma=5.)
        bkg_estimator = MedianBackground()
        bkg = Background2D(data, (50, 50), filter_size=(3, 3),
                        sigma_clip=sigma_clip,
                        bkg_estimator=bkg_estimator)
        data_sub = data - bkg.background
        photometry = aperture_photometry(data_sub, apertures)
        print("displaying photometry")
        display(photometry)


In [ ]:
stars = find_stars(data,
           fwhm=3.0,
           std_multiple=5.,
           photometry=True,
           cmap='Greys',
           vmax=10000)

In [ ]:
# Run this cell to load dataset
# it might take a few seconds
from photutils.datasets import load_spitzer_image

spitzer_image = load_spitzer_image()

In [ ]:
# Run this cell to print the data image
plt.imshow(spitzer_image.data, origin='lower', vmax=65)

In [ ]:
# Remember to analyse the subset of the image first

# Plot subset image
subset = spitzer_image.data[0:400, 200:600]
plt.imshow(subset, origin='lower', vmax=65, cmap='viridis')
plt.show()

In [ ]:
# First try setting fwhm somewhere around 10 and see what you get and go from there
find_stars(subset, 
            fwhm=,
            std_multiple=5,    # comment if using threshold
           # threshold=,       # comment if using std_multiple
           cmap='Greys', 
           vmax=60)

# 6. Use cases of the `DAOStarFinder` algorithm

1. X-ray binary identification. 
2. Cataloging and astrometry of astronomical sources.
3. Identification of point sources in a galaxy.
    1. Identification of star clusters to study gas and star formation -- [Phangs Collaboration](https://sites.google.com/view/phangs/home)
4. Identification of galaxies ([skyviewer](https://skyviewer.app/explorer))
5. Identification and cataloging of asteroids in the asteroid belts.

There's so much science possible with this! With just basic photometry, we can investigate (or estimate) so many additional properties of these astronomical objects, their nature, their mass, their ages, their environment! The possibilities are 'astronomically' endless!